In [ ]:
pip install transformers[torch] accelerate -U pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [ ]:
pip install transformers torch pandas scikit-learn evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which i

In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, Trainer, TrainingArguments, BertModel, BertPreTrainedModel, DataCollatorWithPadding
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/PFA-Dataset/datas/merged_data3.csv')

# Prepare data
captions = df['Captions'].tolist()
labels = df[['O', 'C', 'E', 'A', 'N']].values
train_texts, test_texts, train_labels, test_labels = train_test_split(captions, labels, test_size=0.2, random_state=42, shuffle=True)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

In [ ]:
# Create a PyTorch dataset
class PersonalityDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PersonalityDataset(train_encodings, train_labels)
test_dataset = PersonalityDataset(test_encodings, test_labels)

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel, BertTokenizer

class CustomBertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.1)  # Increased dropout
        self.dense1 = nn.Linear(config.hidden_size, 512)  # Reduced size of dense layer
        self.batch_norm1 = nn.BatchNorm1d(512)  # Batch normalization
        self.activation = nn.ReLU()
        self.dense2 = nn.Linear(512, config.num_labels)  # Output layer

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        x = self.dense1(pooled_output)
        x = self.batch_norm1(x)
        x = self.activation(x)
        logits = self.dense2(x)
        probs = torch.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels)

        output = (probs,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

# Load the tokenizer and custom model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = CustomBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of CustomBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['batch_norm1.bias', 'batch_norm1.num_batches_tracked', 'batch_norm1.running_mean', 'batch_norm1.running_var', 'batch_norm1.weight', 'dense1.bias', 'dense1.weight', 'dense2.bias', 'dense2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = np.array(logits)
    labels = np.array(labels)

    # Apply different thresholds for different labels
    thresholds = np.array([0.6, 0.3, 0.2, 0.6, 0.2])
    predictions = (logits > thresholds).astype(int)

    flat_predictions = predictions.flatten()
    flat_labels = labels.flatten()

    # Ignore the fifth value in the calculations
    flat_predictions = np.delete(flat_predictions, np.arange(4, len(flat_predictions), 5))
    flat_labels = np.delete(flat_labels, np.arange(4, len(flat_labels), 5))

    accuracy = accuracy_score(flat_labels, flat_predictions)
    f1 = f1_score(flat_labels, flat_predictions, average='weighted')

    return {'accuracy': accuracy, 'f1': f1}

# Initialize the BERT model for sequence classification with multi-label classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
model.config.problem_type = "multi_label_classification"

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training arguments with early stopping
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=100,  # Increase epochs for better training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',  # Set save strategy to 'epoch'
    save_total_limit=3,  # Keep only last 3 models
    load_best_model_at_end=True,  # Load best model at end
    metric_for_best_model='eval_loss',  # Use evaluation loss to determine the best model
    greater_is_better=False
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
from keras.utils import plot_model

plot_model(
    model,
    to_file='cnn.png',
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,  # Expand nested models
    rankdir='TB',  # Direction: 'TB' (top to bottom), 'LR' (left to right)
    dpi=300,  # Image resolution
)

NameError: name 'model' is not defined

In [ ]:
# Save the model
torch.save(model.state_dict(), '/content/drive/My Drive/bert.h5')


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

# Define the thresholds for classification
thresholds = np.array([0.6, 0.3, 0.2, 0.6, 0.2])

# Predict using the model
predictions = trainer.predict(test_dataset)
logits = predictions.predictions

# Apply thresholds to obtain binary predictions
preds = (logits > thresholds).astype(int)

# Ignore the fifth value in the predictions and labels
preds = preds[:, :-1]
test_labels = test_labels[:, :-1]

# Print classification report
print(classification_report(test_labels, preds, target_names=['O', 'C', 'E', 'A']))

# Calculate accuracy
accuracy = accuracy_score(test_labels, preds)

# Calculate F1-score
f1 = f1_score(test_labels, preds, average='weighted')

print(f"Accuracy on test set: {accuracy}")
print(f"F1-score on test set: {f1}")

<ipython-input-9-18671b5902fa>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           O       0.82      0.79      0.80        85
           C       0.76      0.81      0.78        63
           E       0.70      0.64      0.67        55
           A       0.85      0.76      0.80        90

   micro avg       0.79      0.75      0.77       293
   macro avg       0.78      0.75      0.76       293
weighted avg       0.79      0.75      0.77       293
 samples avg       0.75      0.74      0.71       293

Accuracy on test set: 0.36764705882352944
F1-score on test set: 0.7723584597672599


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save the best model
model.save_pretrained('/content/drive/MyDrive/model2bert')
tokenizer.save_pretrained('/content/drive/MyDrive/model2bert')

('/content/drive/MyDrive/model2bert/tokenizer_config.json',
 '/content/drive/MyDrive/model2bert/special_tokens_map.json',
 '/content/drive/MyDrive/model2bert/vocab.txt',
 '/content/drive/MyDrive/model2bert/added_tokens.json')

In [ ]:
# Print the predictions for each text
for i, text in enumerate(test_texts[:100]):  # Example: print first 10 texts
    print(f"Text: {text}")
    print(f"Predicted labels: {preds[i]}")
    print(f"True labels: {test_labels[i]}")

Text: ['ready coming next week wait show', 'ready bass face savoring food mayday dortmund rocket', 'mayday dortmund firefirerocketrocket', 'mayday dortmund smiling face hornsred heart', 'smiling face hornsheart hands tonight mayday dortmund video red heart', 'finally surrounded right people make feel home let spread blue heartbutterfly jens thele', 'ever hear blah blah blah face tongue', 'hello sun', 'upcoming week smiling face hornsdown arrow united kingdom switzerland mayday dortmund germany', 'something coming', 'last week eldn starstruckunited kingdom way tonight', 'sold debut eldn firefirefire wait back favorite moment smiling face horns', 'ready new music index pointing viewer light skin tonebeaming face smiling eyesbeaming face smiling eyes', 'hare ram still going strong heart handsfiresmiling face hearteyes', 'birthday girl grinning facebirthday cakeparty popper another amazing year filled much love red heartred heartfolded hands mediumlight skin tonefolded hands mediumlight sk

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Load the saved model and tokenizer
model_path = '/content/drive/MyDrive/model2bert'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Ensure your test_dataset is available
# Assuming test_dataset is already defined and loaded as a dataset object

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Set the model to evaluation mode
model.eval()

predictions = []
true_labels = []

# Perform predictions
with torch.no_grad():
    for batch in test_dataloader:
        # Move tensors to the CPU
        batch = {k: v.to('cpu') for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Apply different thresholds for different labels
        thresholds = np.array([0.7, 0.3, 0.2, 0.6, 0.2])
        preds = (logits.numpy() > thresholds).astype(int)

        predictions.extend(preds)
        true_labels.extend(batch['labels'].numpy())

# Convert lists to arrays for printing
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report
print(classification_report(true_labels, predictions, target_names=['O', 'C', 'E', 'A', 'N']))

# Calculate and print accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

# Print the actual and predicted values for each text
# Assuming you have test_texts as a list of test captions
for i, text in enumerate(test_texts):
    print(f"Text: {text}")
    print(f"Actual labels: {true_labels[i]}")
    print(f"Predicted labels: {predictions[i]}")
    print()

<ipython-input-9-18671b5902fa>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           O       0.82      0.74      0.78        85
           C       0.76      0.81      0.78        63
           E       0.70      0.64      0.67        55
           A       0.85      0.76      0.80        90
           N       0.71      0.91      0.79        32

   micro avg       0.78      0.76      0.77       325
   macro avg       0.77      0.77      0.76       325
weighted avg       0.79      0.76      0.77       325
 samples avg       0.78      0.76      0.74       325

Accuracy: 0.3161764705882353
F1 Score: 0.7681021678239802
Text: ['ready coming next week wait show', 'ready bass face savoring food mayday dortmund rocket', 'mayday dortmund firefirerocketrocket', 'mayday dortmund smiling face hornsred heart', 'smiling face hornsheart hands tonight mayday dortmund video red heart', 'finally surrounded right people make feel home let spread blue heartbutterfly jens thele', 'ever hear blah blah blah face tongue', 'hello s

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import hamming_loss

# Load the saved model and tokenizer
model_path = '/content/drive/MyDrive/model2bert'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Ensure your test_dataset is available
# Assuming test_dataset is already defined and loaded as a dataset object

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Set the model to evaluation mode
model.eval()

predictions = []
true_labels = []

# Perform predictions
with torch.no_grad():
    for batch in test_dataloader:
        # Move tensors to the CPU
        batch = {k: v.to('cpu') for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Apply different thresholds for different labels
        thresholds = np.array([0.7, 0.3, 0.2, 0.6, 0.2])
        preds = (logits.numpy() > thresholds).astype(int)

        predictions.extend(preds)
        true_labels.extend(batch['labels'].numpy())

# Convert lists to arrays for printing
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Print classification report
print(classification_report(true_labels, predictions, target_names=['O', 'C', 'E', 'A', 'N']))

# Calculate and print accuracy and F1 score
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

# Compute Hamming loss
hamming_loss_value = hamming_loss(true_labels, predictions)

print("Hamming Loss:", hamming_loss_value)

# Print the actual and predicted values for each text
# Assuming you have test_texts as a list of test captions
# for i, text in enumerate(test_texts):
#     print(f"Text: {text}")
#     print(f"Actual labels: {true_labels[i]}")
#     print(f"Predicted labels: {predictions[i]}")
#     print()

<ipython-input-9-18671b5902fa>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           O       0.82      0.74      0.78        85
           C       0.76      0.81      0.78        63
           E       0.70      0.64      0.67        55
           A       0.85      0.76      0.80        90
           N       0.71      0.91      0.79        32

   micro avg       0.78      0.76      0.77       325
   macro avg       0.77      0.77      0.76       325
weighted avg       0.79      0.76      0.77       325
 samples avg       0.78      0.76      0.74       325

Accuracy: 0.3161764705882353
F1 Score: 0.7681021678239802
Hamming Loss: 0.21764705882352942


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
